# Анализ спроса на самокаты

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import warnings
warnings.filterwarnings('ignore')

## Предобработка данных о погоде

In [23]:
# Чтение данных и удаление первой строки с единицами измерения
df = pd.read_csv('weather.csv', na_values="", parse_dates=['Datetime']).drop(index=0, axis=1)

# Названия колонок к PEP8
df.columns = df.columns.str.replace(' ', '_').str.lower()

# Приведение колонок к float, кроме колонки с датами
for col in df.columns[1:]:
    df[col] = df[col].astype(float)

### Проверка на дубликаты

In [24]:
df.duplicated().sum(), df['datetime'].duplicated().sum()

(np.int64(0), np.int64(0))

### Работа с пропусками

In [25]:
df.isna().sum() # Проверка наличия пропусков

datetime                0
temperature            55
precipitation_total    30
wind_gust               0
wind_speed             16
cloud_cover_total      20
sunshine_duration      12
dtype: int64

Так как мы работаем с временными рядами, в заполнении пропусков может помочь **интерполяция**, которая отталкивается от значений соседних точек.

#### ДО

In [26]:
df.iloc[1003:1015]

,datetime,temperature,precipitation_total,wind_gust,wind_speed,cloud_cover_total,sunshine_duration
1004,2023-05-12 19:00:00,14.070246,0.1,18.720000,6.130579,28.0,42.074078
1005,2023-05-12 20:00:00,13.690246,0.0,14.400000,5.506941,26.0,0.000000
1006,2023-05-12 21:00:00,NaN,0.0,12.959999,6.569383,20.0,0.000000
1007,2023-05-12 22:00:00,NaN,0.0,14.040000,7.244860,38.0,0.000000
1008,2023-05-12 23:00:00,NaN,0.0,14.040000,6.618519,38.0,0.000000
1009,2023-05-13 00:00:00,NaN,0.0,13.679999,7.729527,25.0,0.000000
1010,2023-05-13 01:00:00,NaN,0.0,12.959999,7.172949,23.7,0.000000
1011,2023-05-13 02:00:00,NaN,0.0,12.599999,8.557102,23.1,0.000000
1012,2023-05-13 03:00:00,NaN,0.0,11.159999,6.725354,59.0,0.000000
1013,2023-05-13 04:00:00,NaN,0.0,10.799999,6.638072,77.0,1.440741


#### ПОСЛЕ

In [27]:
for col in df.columns[1:]:
    df[col] = df[col].interpolate()
    
df.iloc[1003:1015]

,datetime,temperature,precipitation_total,wind_gust,wind_speed,cloud_cover_total,sunshine_duration
1004,2023-05-12 19:00:00,14.070246,0.1,18.720000,6.130579,28.0,42.074078
1005,2023-05-12 20:00:00,13.690246,0.0,14.400000,5.506941,26.0,0.000000
1006,2023-05-12 21:00:00,13.488024,0.0,12.959999,6.569383,20.0,0.000000
1007,2023-05-12 22:00:00,13.285801,0.0,14.040000,7.244860,38.0,0.000000
1008,2023-05-12 23:00:00,13.083579,0.0,14.040000,6.618519,38.0,0.000000
1009,2023-05-13 00:00:00,12.881357,0.0,13.679999,7.729527,25.0,0.000000
1010,2023-05-13 01:00:00,12.679134,0.0,12.959999,7.172949,23.7,0.000000
1011,2023-05-13 02:00:00,12.476912,0.0,12.599999,8.557102,23.1,0.000000
1012,2023-05-13 03:00:00,12.274690,0.0,11.159999,6.725354,59.0,0.000000
1013,2023-05-13 04:00:00,12.072467,0.0,10.799999,6.638072,77.0,1.440741


### Работа со значениями

In [28]:
# Перевод порыва и скорости ветра в м/с и округление до 1 знака после запятой
df[['wind_gust', 'wind_speed']] = (df[['wind_gust', 'wind_speed']] / 3.6).round(1)

# Округление продолжительности солнечного сияния, температуры и облачности до целого
df[['sunshine_duration', 'temperature', 'cloud_cover_total']] = df[['sunshine_duration', 'temperature', 'cloud_cover_total']].round().astype(int)

df.iloc[1234:1240]

,datetime,temperature,precipitation_total,wind_gust,wind_speed,cloud_cover_total,sunshine_duration
1235,2023-05-22 10:00:00,23,0.0,4.6,0.9,8,55
1236,2023-05-22 11:00:00,24,0.0,5.1,1.1,4,58
1237,2023-05-22 12:00:00,25,0.0,5.0,1.4,4,58
1238,2023-05-22 13:00:00,25,0.1,5.0,1.9,9,55
1239,2023-05-22 14:00:00,26,0.6,6.0,1.0,18,49
1240,2023-05-22 15:00:00,25,0.5,6.4,1.1,19,48


### Проверка на выбросы

In [29]:
df.describe()

,datetime,temperature,precipitation_total,wind_gust,wind_speed,cloud_cover_total,sunshine_duration
count,3672,3672.000000,3672.000000,3672.000000,3672.000000,3672.000000,3672.000000
mean,2023-06-16 11:30:00.000000256,18.087691,0.100218,5.801825,2.039434,40.980664,21.543573
min,2023-04-01 00:00:00,-2.000000,0.000000,0.800000,0.000000,0.000000,0.000000
25%,2023-05-09 05:45:00,13.000000,0.000000,3.700000,1.200000,15.000000,0.000000
50%,2023-06-16 11:30:00,18.000000,0.000000,5.100000,1.700000,32.000000,15.000000
75%,2023-07-24 17:15:00,23.000000,0.000000,7.300000,2.600000,67.000000,43.000000
max,2023-08-31 23:00:00,36.000000,5.800000,19.800000,8.300000,100.000000,60.000000
std,NaN,6.838842,0.342384,2.899036,1.184241,31.313242,21.890226


Все значения находятся в норме, выбросов не наблюдается

### Выгрузка новой таблицы

In [30]:
new_units_of_measure = ["", "°C", "mm", "m/s", "m/s", "%", "min"] # Скорости теперь в м / с

first_row = pd.DataFrame([new_units_of_measure], columns=df.columns, index=[0]) 
df = pd.concat([first_row, df], ignore_index=True) # Добавление в начало таблицы


df.to_csv('processed_weather.csv', index=False)

## Предобработка данных о поездках

In [130]:
# Чтение данных и удаление первой строки с единицами измерения
df = pd.read_csv('rides.csv', na_values="", parse_dates=['Start Date', 'End Date'])

# Названия колонок к PEP8
df.columns = df.columns.str.replace(' ', '_').str.lower()
    
# Приводим значения направлений движения к единому формату
df['end_district'] = df['end_district'].str.lower().str.strip().str.replace(" ", '-')
df['start_district'] = df['start_district'].str.lower().str.strip().str.replace(" ", '-')

# Приводим значения начальных и конечных точек к единому формату
df['start_location'] = df['start_location'].str.lower().str.strip().str.replace(".", ' ').str.replace("  ", ' ')
df['start_location'] = df['start_location'].str.replace('ул ', '')

df['end_location'] = df['end_location'].str.lower().str.strip().str.replace(".", ' ').str.replace("  ", ' ')
df['end_location'] = df['end_location'].str.replace('ул ', '')

### Проверка на дубликаты

In [131]:
df.duplicated().sum()

np.int64(4883)

Данные, представленные в таблице rides.csv не должны дублироваться, поэтому удаляем дубликаты

In [132]:
df = df.drop_duplicates()

### Работа с пропусками

In [133]:
df.isna().sum() # Проверка наличия пропусков

id                   0
start_date           0
end_date           555
start_location       0
start_district       0
end_location         0
end_district         0
distance          1181
promo                0
dtype: int64

Наблюдаются пропуски с дате окончания поездки и дистанции поездки.

* Пропуски в end_date будем заполнять на основании пройденной дистанции

* Пропуски в distance будем заполнять на основании времени поездки

#### Работа с пропусками в end_date

Рассчитывать время конца поездки будем на основании медианы средней скорости во время пути и дистанции поездки.

Формула: Конец_поездки = начало_поездки + (дистанция / медиана_средней_скорости)

##### ДО

In [134]:
df.iloc[1015:1018]

,id,start_date,end_date,start_location,start_district,end_location,end_district,distance,promo
1071,101355,2023-04-23 06:19:33,2023-04-23 06:36:02,береговая,центральный,энтузиастов,центральный,2822.00,0
1072,101356,2023-04-23 06:40:06,2023-04-23 07:29:19,озерная,юго-восточный,заречная,юго-восточный,4980.00,0
1073,101357,2023-04-23 06:58:57,2023-04-23 07:18:38,школьная,северо-западный,парковая,юго-восточный,4111.00,0


##### ПОСЛЕ

In [135]:
# Создаем колонну ride_duration, которая содержит длительность поездки в минутах
df['ride_duration'] = df.apply(lambda row: (row.end_date - row.start_date).seconds / 60, axis = 1)

# Создаем колонку average_speed, которая хранит среднюю скорость на протяжении всего пути в км/ч
df['average_speed'] = df.apply(lambda row: (row.distance / 1000) / (row.ride_duration / 60), axis=1)

median_average_speed = df['average_speed'].median()

# Заполнение пустых значений в столбце 'end_date'
df['end_date'] = df['end_date'].fillna(
    df['start_date'] + pd.to_timedelta((df['distance'] / 1000) / median_average_speed, unit='h')
)

df.iloc[1015:1018]

,id,start_date,end_date,start_location,start_district,end_location,end_district,distance,promo,ride_duration,average_speed
1071,101355,2023-04-23 06:19:33,2023-04-23 06:36:02,береговая,центральный,энтузиастов,центральный,2822.00,0,16.48,10.27
1072,101356,2023-04-23 06:40:06,2023-04-23 07:29:19,озерная,юго-восточный,заречная,юго-восточный,4980.00,0,49.22,6.07
1073,101357,2023-04-23 06:58:57,2023-04-23 07:18:38,школьная,северо-западный,парковая,юго-восточный,4111.00,0,19.68,12.53


#### Работа с пропусками в distance

Рассчитывать дистанцию поездки будем на основании медианы средней скорости во время пути и длительности поездки.

Формула: Дистанция = медиана_средней_скорости * время_поездки

##### ДО

In [136]:
df.iloc[1037:1040]

,id,start_date,end_date,start_location,start_district,end_location,end_district,distance,promo,ride_duration,average_speed
1095,101377,2023-04-23 07:48:45,2023-04-23 08:18:40,молодежная,юго-восточный,буденного,октябрьский,4.00,0,29.92,0.01
1096,101378,2023-04-23 07:48:47,2023-04-23 08:17:30,комсомольская,заречный,красноармейская,юго-восточный,4234.00,0,28.72,8.85
1098,101379,2023-04-23 07:49:46,2023-04-23 08:07:27,космонавтов,юго-восточный,парковая,юго-восточный,2943.00,0,17.68,9.99


##### ПОСЛЕ

In [137]:
# Заполнение пустых значений в столбце 'distance'
df['distance'] = df['distance'].fillna(median_average_speed * (df['ride_duration'] / 60) * 1000)

df.iloc[1037:1040]

,id,start_date,end_date,start_location,start_district,end_location,end_district,distance,promo,ride_duration,average_speed
1095,101377,2023-04-23 07:48:45,2023-04-23 08:18:40,молодежная,юго-восточный,буденного,октябрьский,4.00,0,29.92,0.01
1096,101378,2023-04-23 07:48:47,2023-04-23 08:17:30,комсомольская,заречный,красноармейская,юго-восточный,4234.00,0,28.72,8.85
1098,101379,2023-04-23 07:49:46,2023-04-23 08:07:27,космонавтов,юго-восточный,парковая,юго-восточный,2943.00,0,17.68,9.99


#### Проверка работы

In [138]:
df = df.drop(['ride_duration', 'average_speed'], axis=1) # Удаляем уже ненужные стоблцы

df.isna().sum() # Проверка наличия пропусков

id                0
start_date        0
end_date          0
start_location    0
start_district    0
end_location      0
end_district      0
distance          0
promo             0
dtype: int64

### Проверка на выбросы

In [139]:
pd.set_option('display.float_format', '{:.2f}'.format)
df.describe()

,id,start_date,end_date,distance,promo
count,97372.00,97372,97372,97372.00,97372.00
mean,149025.50,2023-06-13 21:06:08.931407360,2023-06-13 21:33:03.576468480,26998.68,0.02
min,100340.00,2023-04-22 09:01:03,2023-04-22 09:24:45,1.00,0.00
25%,124682.75,2023-05-20 21:41:39.500000,2023-05-20 22:07:59,3044.00,0.00
50%,149025.50,2023-06-14 15:15:46,2023-06-14 15:44:53,3703.00,0.00
75%,173368.25,2023-07-08 19:50:22.249999872,2023-07-08 20:18:30,4433.00,0.00
max,197711.00,2023-07-31 23:57:01,2023-08-14 07:25:37.719536260,7569789.00,1.00
std,28109.02,NaN,NaN,307435.12,0.14


В distance присутствуют очень сильные выбросы, вероятно, они вызваны ошибками в показаниях датчиков

Так как выбросы в distance аномально большие или маленькие, то самым простым способом будет вручную отфильтровать выбросы и исправить ошибочные значения, не прибегая к интерквартильному размаху или стандартным отклонениям

Выбросы будем исправлять по аналогии с пропусками.

Некоторые строки с выбросами также содержали пустые данные и на этапе заполнения пустых данных, заполненные пропуски тоже получили некорректные значения. При условии, что один из параметров выброс, а другой был пропущен - мы не можем восстановить данные, следовательно придется их удалить.

In [140]:
# Обновляем значения в колонке ride_duration
df['ride_duration'] = df.apply(lambda row: (row.end_date - row.start_date).seconds / 60, axis=1)

# Удаляем значения, которые невозможно восстановить и обновляем индексацию
df = df[(df['ride_duration'] < 90) & (df['ride_duration'] > 10)]
df = df.reset_index(drop=True)

##### ДО

In [141]:
df.iloc[954:957]

,id,start_date,end_date,start_location,start_district,end_location,end_district,distance,promo,ride_duration
954,101299,2023-04-22 23:54:39,2023-04-23 00:23:58,дачная,центральный,свободы,ленинский,4273850.00,0,29.32
955,101300,2023-04-22 23:58:02,2023-04-23 00:20:51,заречная,юго-восточный,спортивная,октябрьский,3186.00,0,22.82
956,101301,2023-04-22 23:58:44,2023-04-23 00:22:00,подгорная,ленинский,озерная,юго-восточный,4679.00,0,23.27


##### ПОСЛЕ

In [142]:
# Заменяем выбросы в соответствии с формулой
df['distance'] = df['distance'].where(
    (df['distance'] >= 10) & (df['distance'] <= 50000), 
    median_average_speed * (df['ride_duration'] / 60) * 1000
).round().astype(int)

df.iloc[954:957]

,id,start_date,end_date,start_location,start_district,end_location,end_district,distance,promo,ride_duration
954,101299,2023-04-22 23:54:39,2023-04-23 00:23:58,дачная,центральный,свободы,ленинский,4454,0,29.32
955,101300,2023-04-22 23:58:02,2023-04-23 00:20:51,заречная,юго-восточный,спортивная,октябрьский,3186,0,22.82
956,101301,2023-04-22 23:58:44,2023-04-23 00:22:00,подгорная,ленинский,озерная,юго-восточный,4679,0,23.27


#### Проверка работы

In [143]:
df = df.drop(['ride_duration'], axis=1) # Удаляем теперь ненужные колонны таблицы

df.describe()

,id,start_date,end_date,distance,promo
count,96638.00,96638,96638,96638.00,96638.00
mean,149020.97,2023-06-13 20:58:56.413264128,2023-06-13 21:25:26.435115264,3845.82,0.02
min,100340.00,2023-04-22 09:01:03,2023-04-22 09:24:45,1094.00,0.00
25%,124672.25,2023-05-20 21:32:31.750000128,2023-05-20 21:58:06,3119.00,0.00
50%,149018.50,2023-06-14 15:08:45,2023-06-14 15:37:43.500000,3743.00,0.00
75%,173376.75,2023-07-08 19:55:47.500000,2023-07-08 20:21:19.500000,4456.00,0.00
max,197711.00,2023-07-31 23:57:01,2023-08-01 00:42:12,12675.00,1.00
std,28113.79,NaN,NaN,1036.42,0.14


### Выгрузка новой таблицы

In [144]:
df.to_csv('processed_rides.csv', index=False, date_format='%Y-%m-%d %H:%M:%S')

## Объединение таблиц

Для более удобного анализа данных стоит объединить имеющиеся таблицы

In [145]:
rides_df = pd.read_csv('processed_rides.csv', parse_dates=['start_date', 'end_date'])
weather_df = pd.read_csv('processed_weather.csv', parse_dates=['datetime']).drop(index=0, axis=1)

rides_df['rounded_start_time'] = rides_df['start_date'].dt.round('h')


merge_df = pd.merge_asof(
    rides_df.sort_values('rounded_start_time'), 
    weather_df.sort_values('datetime'), 
    left_on='rounded_start_time', 
    right_on='datetime', 
    direction='backward'
)

merge_df = merge_df.drop('rounded_start_time', axis=1)

merge_df.to_csv('rides_&_weather.csv', index=False)

## Проведение расчетов и иссследований

### 1. Расчет итоговой стоимости

![alt text](image.png)

In [ ]:
# тык-тык тут код

### 2. Стимулирует ли акция спрос на самокаты?

. . .

## Проверка гипотез

### 1. В дождливое время спрос на самокаты ниже.

H0: Спрос на самокаты <u>не зависит</u> от погодных условий

H1: Спрос на самокаты <u>зависит</u> от погодных условий

In [ ]:
# тут код с проверкой теории, и объяснением почему мы используем какие-то коэффиценты.